# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298255469009                   -0.85    5.1         
  2   -8.300190722038       -2.71       -1.25    1.2    164ms
  3   -8.300432620934       -3.62       -1.88    2.9    213ms
  4   -8.300461218762       -4.54       -2.77    1.9    185ms
  5   -8.300464381040       -5.50       -3.11    2.8    237ms
  6   -8.300464519946       -6.86       -3.29    4.4    260ms
  7   -8.300464579054       -7.23       -3.45    9.5    654ms
  8   -8.300464613113       -7.47       -3.61    2.2    239ms
  9   -8.300464633635       -7.69       -3.78    1.4    234ms
 10   -8.300464639017       -8.27       -3.90    1.4    242ms
 11   -8.300464643298       -8.37       -4.19    1.6    289ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64170692398                   -0.70    5.9         
  2   -16.67832925963       -1.44       -1.14    1.4    481ms
  3   -16.67921299485       -3.05       -1.88    3.6    580ms
  4   -16.67927984594       -4.17       -2.75    3.2    512ms
  5   -16.67928598596       -5.21       -3.09    4.0    661ms
  6   -16.67928618894       -6.69       -3.50    1.9    429ms
  7   -16.67928621554       -7.58       -3.98    2.0    463ms
  8   -16.67928622152       -8.22       -4.55    2.6    494ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32745560974                   -0.56    7.6         
  2   -33.33454206625       -2.15       -1.00    1.2    1.57s
  3   -33.33598218545       -2.84       -1.72    4.6    2.07s
  4   -33.33616297277       -3.74       -2.63    6.5    2.22s
  5   -33.33687925511       -3.14       -2.40    5.4    2.41s
  6   -33.33690514819       -4.59       -2.52    2.5    1.61s
  7   -33.33694308807       -4.42       -3.67    1.8    1.67s
  8   -33.33694372772       -6.19       -4.01    5.4    2.67s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298680946972                   -0.85    5.2         
  2   -8.300283674127       -2.80       -1.59    1.0    140ms
  3   -8.300435901544       -3.82       -2.62    2.2    171ms
  4   -8.300412809598   +   -4.64       -2.39    7.4    402ms
  5   -8.300464106422       -4.29       -3.36    1.0    181ms
  6   -8.300464592587       -6.31       -3.78    2.8    220ms
  7   -8.300464640394       -7.32       -4.40    1.2    181ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32747703165                   -0.56    7.4         
  2   -33.32880393438       -2.88       -1.28    1.6    1.36s
  3   -24.26189618922   +    0.96       -0.59    6.2    2.56s
  4   -33.26092912265        0.95       -1.55    5.4    2.47s
  5   -33.21895475419   +   -1.38       -1.31    2.6    1.84s
  6   -32.29342825693   +   -0.03       -1.05    4.4    2.14s
  7   -33.32950031575        0.02       -2.05    3.6    1.92s
  8   -33.33471094142       -2.28       -2.31    2.5    1.65s
  9   -33.33559372588       -3.05       -2.41    2.5    1.56s
 10   -33.33600840654       -3.38       -2.53    1.4    1.37s
 11   -33.33644768406       -3.36       -2.68    1.9    1.44s
 12   -33.33690793639       -3.34      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.